# Analisi di Confronto del Mercato Azionario

In [1]:
import pandas as pd
import yfinance as yf
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [2]:
# Define the tickers and market benchmark
tickers = ['AAPL', 'GOOGL', 'MSFT', 'NFLX']
market_ticker = '^GSPC'

# Define the date range
start_date = '2023-07-01'
end_date = '2023-09-30'

# Create a dictionary to store data for each ticker
stock_data = {}

# Fetch historical stock price data using yfinance
for ticker in tickers:
    stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Calculate some information of trickers
for ticker in tickers:
    stock_data[ticker]['Daily_Return'] = stock_data[ticker]['Adj Close'].pct_change() # Daily Returns
    stock_data[ticker]['Cumulative_Return'] = (1 + stock_data[ticker]['Daily_Return']).cumprod() - 1 # Cumulative Returns
    stock_data[ticker]['Volatility'] = stock_data[ticker]['Daily_Return'].std() # Hist Volatility (standard deviation of daily returns)

In [4]:
# Create a figure to visualize the daily returns
fig = go.Figure()

colors = ['blue', 'green', 'yellow', 'red']

for i, ticker in enumerate(tickers):
    fig.add_trace(go.Scatter(x=stock_data[ticker].index, y=stock_data[ticker]['Daily_Return'],
                             mode='lines', name=ticker, line=dict(color=colors[i])))

fig.update_layout(title='Daily Returns for Apple, Google and Microsoft (Last Quarter)',
                  xaxis_title='Date', yaxis_title='Daily Return',
                  legend=dict(x=0.9, y=1.3))

fig.show()

In [5]:
# Create a figure to visualize the cumulative returns
fig1 = go.Figure()

for i, ticker in enumerate(tickers):
    fig1.add_trace(go.Scatter(x=stock_data[ticker].index, y=stock_data[ticker]['Cumulative_Return'],
                             mode='lines', name=ticker, line=dict(color=colors[i])))

fig1.update_layout(title='Cumulative Returns for Apple, Google and Microsoft (Last Quarter)',
                  xaxis_title='Date', yaxis_title='Cumulative Return',
                  legend=dict(x=0.9, y=1.3))

fig1.show()

In [6]:
# Create a figure to compare volatility
fig2 = go.Figure()

for i, ticker in enumerate(tickers):
    fig2.add_bar(x=[ticker], y=[stock_data[ticker]['Volatility'].iloc[0]],
                 text=[f'{stock_data[ticker]["Volatility"].iloc[0]:.4f}'],
                 textposition='auto', marker=dict(color=colors[i]))

fig2.update_layout(title='Volatility Comparison (Last Quarter)',
                   xaxis_title='Stock', yaxis_title='Volatility (Standard Deviation)',
                   bargap=0.5)
fig2.show()

""""
We first calculated the historical volatility for both Apple and Google stocks. Volatility is a measure of how much the stock’s price 
fluctuates over time. In this case, we are calculating the standard deviation of daily returns to measure the volatility. 
Then we visualized the calculated volatility to assess and compare the volatility or risk associated with both Apple and Google stocks 
during the specified period. We can see that Google’s volatility is higher than Apple’s.

It indicates that Google’s stock price experienced larger price fluctuations or greater price variability over the last quarter. 
Here’s what this difference in volatility may indicate:
1.Google’s stock is considered riskier compared to Apple. Investors generally associate higher volatility with higher risk because it 
implies that the stock price can change significantly in a short period.

2.Google’s stock may be more sensitive to market conditions, economic factors, or company-specific news and events. 
This heightened sensitivity can result in larger price swings.

3.Traders and investors with a higher risk tolerance might find Google’s stock appealing if they are looking for opportunities to 
profit from short-term price movements.
"""

'"\nWe first calculated the historical volatility for both Apple and Google stocks. Volatility is a measure of how much the stock’s price \nfluctuates over time. In this case, we are calculating the standard deviation of daily returns to measure the volatility. \nThen we visualized the calculated volatility to assess and compare the volatility or risk associated with both Apple and Google stocks \nduring the specified period. We can see that Google’s volatility is higher than Apple’s.\n\nIt indicates that Google’s stock price experienced larger price fluctuations or greater price variability over the last quarter. \nHere’s what this difference in volatility may indicate:\n1.Google’s stock is considered riskier compared to Apple. Investors generally associate higher volatility with higher risk because it \nimplies that the stock price can change significantly in a short period.\n\n2.Google’s stock may be more sensitive to market conditions, economic factors, or company-specific news and

In [7]:
# Fetch historical stock price data using yfinance for the market benchmark
market_data = yf.download(market_ticker, start=start_date, end=end_date)
market_data['Daily_Return'] = market_data['Adj Close'].pct_change()

# Calculate Beta for each ticker
beta_values = {}
var_market = market_data['Daily_Return'].var()

for ticker in tickers:
    cov_ticker = stock_data[ticker]['Daily_Return'].cov(market_data['Daily_Return'])
    beta_values[ticker] = cov_ticker / var_market

# Find the ticker with the highest Beta
max_beta_ticker = max(beta_values, key=beta_values.get)

# Conclusion
conclusion = f"{max_beta_ticker} is more volatile (higher Beta) compared to the others."

# Print Beta values and conclusion
for ticker in tickers:
    print(f"Beta for {ticker}: {beta_values[ticker]:.4f}")

print(conclusion)

[*********************100%%**********************]  1 of 1 completed

Beta for AAPL: 1.2257
Beta for GOOGL: 1.5303
Beta for MSFT: 1.3028
Beta for NFLX: 1.8275
NFLX is more volatile (higher Beta) compared to the others.


## Stock Market Analysis into a Dashboard

In [13]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as pc

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Stock Market Analysis Dashboard", className="text-center mb-4"),
    
    # Dropdown for selecting the chart
    dcc.Dropdown(
        id='chart-type-dropdown',
        options=[
            {'label': 'Daily Returns', 'value': 'daily_returns'},
            {'label': 'Cumulative Returns', 'value': 'cumulative_returns'},
            {'label': 'Volatility Comparison', 'value': 'volatility_comparison'},
        ],
        value='daily_returns',
        className="mb-4",
    ),
    
    # Graph container for Stock Chart (fig and fig1)
    html.Div([
        dcc.Graph(id='stock-chart', className="mb-4"),
    ], id='stock-chart-container'),
    
    # Graph container for Volatility Comparison (fig2)
    html.Div([
        dcc.Graph(id='volatility-chart', className="mb-4"),
    ], id='volatility-chart-container'),
])

# Define callback to update the Stock Chart (fig and fig1) and Volatility Chart (fig2)
@app.callback(
    [Output('stock-chart-container', 'style'),
     Output('volatility-chart-container', 'style'),
     Output('stock-chart', 'figure'),
     Output('volatility-chart', 'figure')],
    [Input('chart-type-dropdown', 'value')]
)
def update_charts(selected_chart_type):
    fig = go.Figure()
    fig2 = go.Figure()

    colors = ['blue', 'green', 'yellow', 'red']

    for i, ticker in enumerate(tickers):
        if selected_chart_type == 'daily_returns':
            y_data = stock_data[ticker]['Daily_Return']
            y_label = 'Daily Return'
        elif selected_chart_type == 'cumulative_returns':
            y_data = stock_data[ticker]['Cumulative_Return']
            y_label = 'Cumulative Return'
        else:  # 'volatility_comparison'
            y_data = [stock_data[ticker]['Volatility'].iloc[0]]
            y_label = 'Volatility (Standard Deviation)'
            fig2.add_bar(x=[ticker], y=y_data,
                         text=[f'{stock_data[ticker]["Volatility"].iloc[0]:.4f}'],
                         textposition='auto', marker=dict(color=colors[i]))

        fig.add_trace(go.Scatter(x=stock_data[ticker].index, y=y_data,
                                 mode='lines', name=ticker, line=dict(color=colors[i])))

    fig.update_layout(
        title=f'{selected_chart_type.capitalize()} for Selected Stocks',
        xaxis_title='Date',
        yaxis_title=y_label,
        legend=dict(x=0.9, y=1.3)
    )

    if selected_chart_type == 'volatility_comparison':
        return {'display': 'none'}, {'display': 'block'}, fig, fig2
    else:
        return {'display': 'block'}, {'display': 'none'}, fig, {'data': [], 'layout': {}}

if __name__ == '__main__':
    app.run_server(port=8052)
